In [5]:
import codecs
import os

def read_in(folder):
    files = os.listdir(folder)
    a_list = []
    for a_file in files:
        if not a_file.startswith("."):
            f = codecs.open(os.path.join(folder, a_file), "r", encoding = "ISO-8859-1", errors="ignore")
            a_list.append(f.read())
            f.close()

    return a_list

In [8]:
spam_list = read_in("enron1/spam")
ham_list = read_in("enron1/ham")

In [11]:
import random

all_emails = [(email_content, "spam") for email_content in spam_list]
all_emails += [(email_content, "ham") for email_content in ham_list]
random.seed(42)
random.shuffle(all_emails)
print(f"Dataset size = {len(all_emails)} emails")

Dataset size = 5172 emails
